In [5]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [145]:
query = """SELECT mapbox_account_id, vertical
FROM (SELECT an_mbx_v.*, max(dt) over (partition by 'dummy') as max_dt from analytics.mbx_account_verticals an_mbx_v)
where dt = max_dt
"""

In [146]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
cte_queries = formatter.parse_cte(formatted_query)
cte_queries

{'main': "SELECT mapbox_account_id,\n       vertical\nFROM\n  (SELECT an_mbx_v.*,\n          max(dt) OVER (PARTITION BY 'dummy') AS max_dt\n   FROM analytics.mbx_account_verticals an_mbx_v)\nWHERE dt = max_dt"}

In [147]:
db_fields = pd.DataFrame({'db_table': 'analytics.mbx_account_verticals', 
            'all_columns': ['mapbox_account_id', 'email', 'salesforce_account_id', 'vertical', 'vertical_source', 'dt']})


In [148]:
cte_queries

{'main': "SELECT mapbox_account_id,\n       vertical\nFROM\n  (SELECT an_mbx_v.*,\n          max(dt) OVER (PARTITION BY 'dummy') AS max_dt\n   FROM analytics.mbx_account_verticals an_mbx_v)\nWHERE dt = max_dt"}

In [149]:
query = cte_queries['main']
query.split('\n')

['SELECT mapbox_account_id,',
 '       vertical',
 'FROM',
 '  (SELECT an_mbx_v.*,',
 "          max(dt) OVER (PARTITION BY 'dummy') AS max_dt",
 '   FROM analytics.mbx_account_verticals an_mbx_v)',
 'WHERE dt = max_dt']

In [153]:
e = query.split('\n')[3]

print(sum(list(map(lambda x: '*' in x, re.findall(r"([a-z].*[*])", e)))))
print(re.findall(r"([a-z].*[*])", e))
print(sum(list(map(lambda x: '*' in x, re.findall(r"([*])", e)))))
print(re.findall(r"([*])", e))


1
['an_mbx_v.*']
1
['*']


In [132]:
re.findall(r"([a-z].*[*])", "  b.* FROM")

['b.*']

In [133]:
re.findall(r"([*])", "    (SELECT *,")

['*']

In [167]:
# _get_all_variables
# def _get_all_variables(self, query):
#     """
#     Get all variables including: table names, aliases, column names and aliases, and all other non-sql reserved words.
#     Args:
#         param (string): A string of any type of complete query; allows only complete query but can nest with CTE's and/or subqueries.
#     Returns:
#         list: A list of all variables within the query.
#     """
all_variables = []

for e in query.split('\n'):
    if sum(list(map(lambda x: '*' in x, re.findall(r"([a-z].*[*])", e)))):
        variable = re.findall(r"([a-z].*[*])", e)
    elif sum(list(map(lambda x: '*' in x, re.findall(r"([*])", e)))):
        variable = re.findall(r"([*])", e)
    else:
        variable = [x.strip(' ') for x in re.findall(r"[a-z_\s.]+", e)]

    all_variables.extend(variable)

#     return all_variables

In [168]:
all_variables

['mapbox_account_id',
 'vertical',
 'an_mbx_v.*',
 'max',
 'dt',
 '',
 '',
 '',
 '',
 'dummy',
 '',
 'max_dt',
 '',
 'analytics.mbx_account_verticals an_mbx_v',
 'dt',
 'max_dt']

In [171]:
# _get_all_scanned_cols
all_columns_scanned = []

cte_query = cte_queries['main']
meta_cols = db_fields

table_alias_mapping = formatter.get_table_names(cte_query.split('\n'))
# variables = formatter._get_all_variables(cte_query)
variables = all_variables
queried_columns = formatter._get_queried_columns(table_alias_mapping, meta_cols)
print(table_alias_mapping)
print(variables)
print(queried_columns)


{'an_mbx_v': 'analytics.mbx_account_verticals'}
['mapbox_account_id', 'vertical', 'an_mbx_v.*', 'max', 'dt', '', '', '', '', 'dummy', '', 'max_dt', '', 'analytics.mbx_account_verticals an_mbx_v', 'dt', 'max_dt']
[{'analytics.mbx_account_verticals': {'vertical_source', 'dt', 'email', 'mapbox_account_id', 'salesforce_account_id', 'vertical'}}]


In [193]:
for var in var_list:
    var_split = var.split('.')
    print(var_split, len(var_split))
    if var_split[0] in table_alias_mapping.keys():
        db_table = table_alias_mapping[var_split[0]]
        print(db_table)
#     else:
#         print(var_split)
    

['mapbox_account_id'] 1
['vertical'] 1
['an_mbx_v', '*'] 2
analytics.mbx_account_verticals
['max'] 1
['dt'] 1
[''] 1
[''] 1
[''] 1
[''] 1
['dummy'] 1
[''] 1
['max_dt'] 1
[''] 1
['analytics', 'mbx_account_verticals an_mbx_v'] 2
['dt'] 1
['max_dt'] 1


In [187]:
if var_split[0] in table_alias_mapping.keys():
    db_table = table_alias_mapping[var_split[0]]
    print(db_table)
    

In [182]:
queried_cols

[{'analytics.mbx_account_verticals': {'dt',
   'email',
   'mapbox_account_id',
   'salesforce_account_id',
   'vertical',
   'vertical_source'}}]

In [184]:
# for db_table_col in queried_cols:
db_table_col = queried_cols[0]
for k,v in db_table_col.items():
    if k == db_table and var_split[1] in v:
        original_columns_list.append("{}.{}".format(k, var_split[1]))
            

In [185]:
original_columns_list

['analytics.mbx_account_verticals.dt',
 'analytics.mbx_account_verticals.mapbox_account_id',
 'analytics.mbx_account_verticals.vertical']

In [ ]:
# var_split = var.split('.')

# if len(var_split) == 1:
#     for db_table in queried_cols:
#         for k,v in db_table.items():
#             if var in v:
#                 original_columns_list.append("{}.{}".format(k, var))

# elif len(var_split) == 2:

   

In [219]:
# _map_db_columns
var_list, queried_cols, table_alias_mapping = variables, queried_columns, table_alias_mapping


original_columns_list = []

for var in set(var_list):

    var = var.strip(' ')

    if var in table_alias_mapping.keys():
        pass

    else:
        var_split = var.split('.')

        if len(var_split) == 1:
            for db_table in queried_cols:
                for k,v in db_table.items():
                    if var in v:
                        original_columns_list.append("{}.{}".format(k, var))

        elif len(var_split) == 2:

            if var_split[0] in table_alias_mapping.keys():
                db_table = table_alias_mapping[var_split[0]]
               
                for db_table_col in queried_cols:
                    for k,v in db_table_col.items():

                        if k == db_table and var_split[1] in v:
                            original_columns_list.append("{}.{}".format(k, var_split[1]))
                        elif k == db_table and var_split[1] == '*':
                            for col in v:
                                original_columns_list.append("{}.{}".format(k, col))     


analytics.mbx_account_verticals {'vertical_source', 'dt', 'email', 'mapbox_account_id', 'salesforce_account_id', 'vertical'}


In [221]:
list(set(original_columns_list))

['analytics.mbx_account_verticals.salesforce_account_id',
 'analytics.mbx_account_verticals.mapbox_account_id',
 'analytics.mbx_account_verticals.vertical_source',
 'analytics.mbx_account_verticals.vertical',
 'analytics.mbx_account_verticals.dt',
 'analytics.mbx_account_verticals.email']

In [ ]:
all_columns_scanned = []
# if variables == []:
#     original_columns_list = []
#     for table in queried_columns:
#         for k,v in table.items():
#             for t in v:
#                 original_columns_list.append("{}.{}".format(k,t))
# else:
original_columns_list = formatter._map_db_columns(variables, queried_columns, table_alias_mapping)
        
all_columns_scanned.extend(list(set(original_columns_list)))


In [164]:
# all_columns_scanned = _get_all_scanned_cols(cte_queries, db_fields)
all_columns_scanned


['analytics.mbx_account_verticals.salesforce_account_id',
 'analytics.mbx_account_verticals.mapbox_account_id',
 'analytics.mbx_account_verticals.vertical_source',
 'analytics.mbx_account_verticals.vertical',
 'analytics.mbx_account_verticals.dt',
 'analytics.mbx_account_verticals.email']

In [158]:
column_payload = []
for column in all_columns_scanned:
    try:
        col_split = column.split('.')
        db, table, col = col_split[0], col_split[1], col_split[2]
        row_payload = dict()
        row_payload["database_name"] = db
        row_payload["table_name"] = table 
        row_payload["column_name"] = col
        for arg, value in kwargs.items():
            row_payload[arg] = value    

        column_payload.append(row_payload)  
    except:
        pass

In [159]:
column_payload

[]